In [275]:
import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column
import scipy
from numpy import histogram, linspace
from scipy.stats import norm
from scipy.stats.kde import gaussian_kde
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tqdm

Loading BokehJS ...

Useful links:

https://www.daxpatterns.com/basket-analysis/
    
https://towardsdatascience.com/use-algorithms-to-recommend-items-to-customers-in-python-347b769b21f3

In [2]:
%%bash
ls -ltr ../../Documents/Data/orders_filled.xlsx

-rw-r--r--@ 1 apple  staff  8588640 Mar 12 07:53 ../../Documents/Data/orders_filled.xlsx


In [3]:
%%time
df = pd.read_excel('../../Documents/Data/orders_filled.xlsx', sheet_name='Sheet1')

CPU times: user 20.9 s, sys: 184 ms, total: 21.1 s
Wall time: 21.1 s


In [4]:
df.head()

,id,Name,fmbgl,Discount Code,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem sku,Billing Province,Billing Country,Shipping Zip,Shipping Province,Shipping Country,Payment Method,Payment Reference,Id,Risk Level,Source
0,17065,#Gvahdam3579,07blfxb@cpmcbst.nft,HELLO20,5-8 Days Delivery | Express Shipping | Extensi...,2019-02-27 00:40:15 +0530,1,High Mountain Darjeeling Second Flush Oolong T...,High Mountain Oolong Tea,CA,US,'20180,VA,US,PayPal Payments Standard,#G02099999.1,966000000000,Low,Excelify.io App
1,17066,#Gvahdam3579,07blfxb@cpmcbst.nft,HELLO20,5-8 Days Delivery | Express Shipping | Extensi...,2019-02-27 00:40:15 +0530,5,India's Original Masala Chai Tea - 3.53oz | 10...,Indias Original Masala Chai,CA,US,'20180,VA,US,PayPal Payments Standard,#G02099999.1,966000000000,Low,Excelify.io App
2,17067,#Gvahdam3579,07blfxb@cpmcbst.nft,HELLO20,5-8 Days Delivery | Express Shipping | Extensi...,2019-02-27 00:40:15 +0530,1,Tea Master's Private Reserve DUO -2 Teas,Tea Masters Private Reserve DUO (2 Tin Caddy),CA,US,'20180,VA,US,PayPal Payments Standard,#G02099999.1,966000000000,Low,Excelify.io App
3,31669,#G02098859,08tbvbg@gmbgl.cpm,HELLO20,5-8 Days Delivery | Express Shipping | Extensi...,2018-09-08 15:13:48 +0530,2,Golden Orange Pekoe Darjeeling Black Tea - 3.5...,RS2692,NY,US,'10017,NY,US,PayPal Payments Standard,#G02098859.1,710000000000,Low,Excelify.io App
4,31670,#G02098859,08tbvbg@gmbgl.cpm,HELLO20,5-8 Days Delivery | Express Shipping | Extensi...,2018-09-08 15:13:48 +0530,1,Classic English Breakfast Black Tea - 3.53oz |...,NSB1,NY,US,'10017,NY,US,PayPal Payments Standard,#G02098859.1,710000000000,Low,Excelify.io App


In [5]:
items = np.unique(df['Lineitem name'])
print "There are about:",items.shape[0], "unique items"

There are about: 3365 unique items


In [185]:
agg_df = df.groupby('Lineitem name') \
        .agg({'Lineitem name': 'count', 'Lineitem quantity':'sum', 'fmbgl':pd.Series.nunique})  \
        .rename(columns={'Lineitem name': 'num_times_sold', 'Lineitem quantity':'tot_qty_sold', 
                         'fmbgl':'num_unq_cust'})
agg_df = agg_df.reset_index().rename(columns={'Lineitem name':'ItemName'})

In [186]:
# clean_df.columns = ["_".join(x) for x in clean_df.columns.ravel()]
agg_df.sort_values(by=['tot_qty_sold'], ascending=False).head()

,ItemName,num_unq_cust,num_times_sold,tot_qty_sold
3269,Turmeric Ginger Herbal Tea Tisane - 3.53oz | 1...,2483,2537,6028
1043,Earl Grey Citrus Black Tea - 3.53oz | 100gm | ...,836,853,4840
756,Classic English Breakfast Black Tea - 3.53oz |...,766,785,4470
706,Chai Tea Private Reserve - 1 Box,3037,3085,3860
3104,Tea Bags Tea Assortment,2273,2285,3632


In [187]:
n = 5
m = 100
sold_n_times_df = agg_df[agg_df['num_times_sold'] > n]
sold_n_times_df = sold_n_times_df[sold_n_times_df['num_times_sold'] < m]
sold_n_times_df.shape
print "Items sold between {0}-{1} times: {2} Items".format(n, m, sold_n_times_df.shape[0])
sold_n_times_df.sort_values(by=['tot_qty_sold'], ascending=False).head()

Items sold between 5-100 times: 714 Items


,ItemName,num_unq_cust,num_times_sold,tot_qty_sold
2038,Lopchu Golden Orange Pekoe Darjeeling Second F...,66,85,183
2021,Lopchu Flowery Orange Pekoe Darjeeling Second ...,79,97,168
701,Castleton Premium Darjeeling First Flush Black...,83,96,161
754,Classic English Breakfast Black Tea - 3.53oz |...,71,92,153
1550,Gopaldhara Wonder Darjeeling First Flush Black...,73,82,144


In [188]:
sold_n_times_df.head()

,ItemName,num_unq_cust,num_times_sold,tot_qty_sold
9,2017 First Flush Spring Tea Sampler (15 Teas) ...,27,27,27
10,2017 First Flush Spring Tea Sampler (19 Teas) ...,10,10,11
11,2017 First Flush Spring Tea Sampler (6 Teas) -...,19,20,27
20,Antu Valley Flowery Pekoe Nepal Second Flush G...,19,20,25
28,Antu Valley Golden Tippy Souchong Nepal Second...,24,30,37


In [252]:
more_then_m = 99
sold_m_times_df = agg_df[agg_df['num_times_sold'] > more_then_m]
print "Items sold more then {0} times: {1} Items".format(more_then_m, sold_m_times_df.shape[0])
sold_m_times_df.sort_values(by=['tot_qty_sold'], ascending=False)

Items sold more then 99 times: 50 Items


,ItemName,num_unq_cust,num_times_sold,tot_qty_sold
3269,Turmeric Ginger Herbal Tea Tisane - 3.53oz | 1...,2483,2537,6028
1043,Earl Grey Citrus Black Tea - 3.53oz | 100gm | ...,836,853,4840
756,Classic English Breakfast Black Tea - 3.53oz |...,766,785,4470
706,Chai Tea Private Reserve - 1 Box,3037,3085,3860
3104,Tea Bags Tea Assortment,2273,2285,3632
1807,India's Original Masala Chai Tea - 3.53oz | 10...,606,607,3545
761,Classic Tea Infuser,2549,2581,3200
453,BLUSH Assorted Teas Gift Box -3 Teas,1429,1465,2010
1711,Himalayan Green Tea - 3.53oz | 100gm | 40+ Cup...,1570,1593,1970
1665,High Mountain Darjeeling Second Flush Oolong T...,1234,1253,1460


In [190]:
def pdf_series(series):
    return gaussian_kde(series)

In [191]:
def hist(series, x_axis_label, y_axis_label):
    x = linspace(n,m,100)
    
    p = figure(x_axis_type = 'log', title = 'Between {0}-{1}: num_times_sold'.format(n,m),
               plot_height=300, plot_width=900,
               x_axis_label= x_axis_label, 
               y_axis_label= y_axis_label)
    p.line(x, pdf_series(series)(x))

    # plot actual hist for comparison
    hist, edges = histogram(series, density=True, bins=100)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4)

    return p

In [192]:
p1 = hist(sold_n_times_df.tot_qty_sold, 'Total Qty Sold', 'P(Total Qty Sold)')
p2 = hist(sold_n_times_df.num_times_sold, 'Num Times Sold', 'P(Num Times Sold)')
p3 = hist(sold_n_times_df.num_unq_cust, 'Num Unique Customers', 'P(Num Unique Customers)')

show(column(p1, p2, p3))

In [262]:
def plot_dist(df, title):
    hover = HoverTool(
            tooltips=[
            ("Item", "@ItemName"),
            ("Times Qty Sold", "@tot_qty_sold"),
            ("Distinct Customers", "@num_unq_cust"),
            ("Distinct Sales", "@num_times_sold")
            ]
        )
    hover.mode = 'mouse'

    p = figure(plot_width=900, plot_height=600, 
               x_axis_label='Distinct Sales', 
               y_axis_label='Distinct Customer', 
               title= title+'Distribution of Sales vs Unique Customers')
    p.tools.append(hover)
    source = ColumnDataSource(df)

    df['log_total_qty_sold'] = np.log(df['tot_qty_sold'])*3

    p.circle('num_times_sold', 'num_unq_cust', 
             source = source,
             size='log_total_qty_sold', 
             line_color="navy", 
             fill_color="orange", fill_alpha=0.5)

    return p

In [263]:
c1 = plot_dist(sold_m_times_df, 'HIGH SALES - ')
c2 = plot_dist(sold_n_times_df, 'MEDIUM SALES - ')

show(column(c1, c2))

### Item Affinity Score Analysis

In [276]:
import multiprocessing
print multiprocessing.cpu_count()

8


In [285]:
%%time

start = datetime.now()


#Get list of unique items
sold_n_or_more = agg_df[agg_df['num_times_sold'] > m]
itemList=list(set(sold_n_or_more["ItemName"].tolist()))

#Get count of users
userCount=len(set(df["fmbgl"].tolist()))

#Create an empty data frame to store item affinity scores for items.
itemAffinity= pd.DataFrame(columns=('item1', 'item2', 'score'))
rowCount=0


print "Started Processing at: ", start
#For each item in the list, compare with other items.
for ind1 in tqdm(range(len(itemList))):
    
    #Get list of users who bought this item 1.
    item1Users = df[df['Lineitem name']==itemList[ind1]]["fmbgl"].tolist()
#     print("Item 1 ", item1Users)
    
    #Get item 2 - items that are not item 1 or those that are not analyzed already.
    for ind2 in range(ind1, len(itemList)):
        
        if ( ind1 == ind2):
            continue
       
        #Get list of users who bought item 2
        item2Users=df[df['Lineitem name']==itemList[ind2]]["fmbgl"].tolist()
#         print("Item 2",item2Users)
        
        #Find score. Find the common list of users and divide it by the total users.
        commonUsers= len(set(item1Users).intersection(set(item2Users)))
        score=commonUsers / userCount

        #Add a score for item 1, item 2
        itemAffinity.loc[rowCount] = [itemList[ind1],itemList[ind2],score]
        rowCount +=1
        #Add a score for item2, item 1. The same score would apply irrespective of the sequence.
        itemAffinity.loc[rowCount] = [itemList[ind2],itemList[ind1],score]
        rowCount +=1
        
#     print "Processed item: ", rowCount, " at: ", (datetime.now() - start).total_seconds()
#     if rowCount > 4500:
#         break
        


Started Processing at:  2019-03-15 11:56:57.480227


CPU times: user 16.7 s, sys: 215 ms, total: 16.9 s
Wall time: 16.8 s


In [286]:
#Check final result
itemAffinity.sort_values(by='score', ascending=False).head()

,item1,item2,score
0,India's Original Masala Chai Tea - 3.53oz | 100gm,Turmeric Ginger Herbal Tea Tisane - 3.53oz | 1...,0
1636,Makaibari Premium Darjeeling First Flush Organ...,Giddapahar Supreme Darjeeling Second Flush Bla...,0
1629,Earl Grey Citrus Black Tea - 3.53oz | 100gm | ...,Makaibari Premium Darjeeling First Flush Organ...,0
1630,Makaibari Premium Darjeeling First Flush Organ...,Himalayan Green Tea - 3.53oz | 100gm | 40+ Cup...,0
1631,Himalayan Green Tea - 3.53oz | 100gm | 40+ Cup...,Makaibari Premium Darjeeling First Flush Organ...,0


In [ ]:
searchItem=5001
recoList=itemAffinity[itemAffinity.item1==searchItem]\
        [["item2","score"]]\
        .sort_values("score", ascending=[0])
        
print("Recommendations for item 5001\n", recoList)